# Visualize ply files (used it when i calculated uncertainty by comparing to gt using 2015's paper equations)

In [ ]:
import open3d as o3d
import sys
import argparse

def visualize_ply(file_path):
    pcd = o3d.io.read_point_cloud(file_path)
    o3d.visualization.draw_geometries([pcd])

visualize_ply("uncertainty_completion_3.ply")
visualize_ply("uncertainty_gt_3.ply")
visualize_ply("overlay_banded_3.ply")

'''
GT range (normalized): [-0.00762572 -0.05296077 -0.42857869] [ 0.04921851  0.01721037 -0.36634152]
Completion range (normalized): [-0.22702396 -0.02778991  0.367522  ] [0.27411154 0.05768301 0.42968768]
Raw completion range: [-0.14732665 -0.01088524  0.4547647 ] [0.15058716 0.0399265  0.4917208 ]
Saved: uncertainty_gt.ply
Saved: uncertainty_completion.ply
[GT] Uncertainty stats:
  Mean: 0.000585
  Min : 0.000418
  Max : 0.001467
  Std : 0.000114

[Completion] Uncertainty stats:
  Mean: 0.999994
  Min : 0.999797
  Max : 1.000000
  Std : 0.000021
'''

# Visualize overlay of partial and completion

In [ ]:
'''
This code processes and visualizes 3D point cloud data stored in `.npz` files. Here’s a summary of how it works:

1. **File Handling**:  
    It looks for files (here, just `"batch0_sample0_data.npz"`) containing point cloud data.

2. **Data Loading**:  
    For each file, it loads arrays like `src_pcd` (source point cloud) and `xyz` (predicted/interpolated points).

3. **Downsampling**:  
    If the predicted point cloud has more than 512 points, it uses Open3D’s `farthest_point_down_sample` to reduce it to 512 points. If fewer, it repeats points to reach 512.

4. **Visualization**:  
    It visualizes the original and predicted point clouds together using Open3D, coloring them differently for comparison.

5. **Normal Calculation**:  
    There’s a function to estimate normals for a point cloud, though it’s commented out in the visualization step.
'''

import glob
import os
import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans
 
import glob, os
import open3d as o3d
import numpy as np


def cal_normal(pcd, radius=0.03, max_nn=30):
    _pcd = o3d.geometry.PointCloud()
    _pcd.points = o3d.utility.Vector3dVector(pcd)
    
    _pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
    # o3d.geometry.estimate_normals(_pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
    normals = np.asarray(_pcd.normals)
    return normals
 
 
def visualize_comparison(src_pcd, src_pcd_inter, batch_idx, sample_idx):
    # Create visualizer
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=f"Batch {batch_idx} Sample {sample_idx}")
    
    # Add source point cloud (red)
    pcd_src = o3d.geometry.PointCloud()
    pcd_src.points = o3d.utility.Vector3dVector(src_pcd)
    pcd_src.paint_uniform_color([1, 0, 0])  # Red
    vis.add_geometry(pcd_src)
    
    # Add interpolated point cloud (blue)
    pcd_inter = o3d.geometry.PointCloud()
    pcd_inter.points = o3d.utility.Vector3dVector(src_pcd_inter)
    pcd_inter.paint_uniform_color([0, 0, 1])  # Blue
    vis.add_geometry(pcd_inter)
    
    # Run visualization
    vis.run()
    vis.destroy_window()
 
 
# Process each file in the directory
 
# base_path = "log/PointAttN_cd_debug_pcn/all"
# files = glob.glob(os.path.join(base_path, "batch*_sample*_data.npz"))
files = ["./all_cd_t_coarse/batch0_sample0_data.npz"]

for file_path in files:
    # Extract batch and sample numbers from filename
    filename = os.path.basename(file_path)
    batch_idx = int(filename.split('_')[0].replace('batch', ''))
    
    sample_idx = int(filename.split('_')[1].replace('sample', ''))
    
    # Load the data
    data = np.load(file_path)
    src_pcd = data['src_pcd']
    src_pcd_pred = data['xyz']
    unique_points = np.unique(src_pcd_pred, axis=0)
    print(f"Number of unique points: {len(unique_points)}, Total points: {len(src_pcd_pred)}")
    #FPS on src_pcd_inter to get 512 points
    pcd_inter = o3d.geometry.PointCloud()
    pcd_inter.points = o3d.utility.Vector3dVector(src_pcd_pred)
    if len(src_pcd_pred) > 512:
        downsampled_pcd_inter = pcd_inter.farthest_point_down_sample(512)
        src_pcd_pred = np.asarray(downsampled_pcd_inter.points)
    else:
        src_pcd_pred = np.asarray(pcd_inter.points)
        repeats_needed = int(np.ceil(512 / len(src_pcd_pred)))
        src_pcd_pred = np.tile(src_pcd_pred, (repeats_needed, 1))[:512]
    
    print(f"Visualizing batch {batch_idx}, sample {sample_idx}")
    #src_pcd_normal = cal_normal(src_pcd_pred)
    visualize_comparison(src_pcd, src_pcd_pred, batch_idx, sample_idx)
    

# Visualize .npz prediction vs dropout

In [ ]:
'''
'src_pcd': inputs.transpose(2, 1)[j].cpu().numpy(),
'src_pcd_inter': result_dict['out1'][j].cpu().numpy(),
'xyz': result_dict['out2'][j].cpu().numpy(),
'model_pcd_transformed': gt[j].cpu().numpy(),
'mean_out2': mean_out2[j].cpu().numpy(),
'std_out2': std_out2[j].cpu().numpy(),
'''


import os
import glob
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

def visualize_single_cloud(points, title, color=None, colormap=False):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    if colormap:
        # For std_out2: visualize uncertainty as viridis color map
        magnitudes = np.linalg.norm(points, axis=1)
        normalized = (magnitudes - magnitudes.min()) / (magnitudes.max() - magnitudes.min() + 1e-8)
        cmap = plt.get_cmap("viridis")(normalized)[:, :3]
        pcd.colors = o3d.utility.Vector3dVector(cmap)
    else:
        pcd.paint_uniform_color(color)

    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=title)
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()

# File(s) to visualize
# files = glob.glob("./all_T60_0.1/batch*_sample*_data.npz")
# files = ["./all_0.1_train_NN_T20.npz"]
# files = ["./all_0.1_sim_train_NN/batch5_sample2_data.npz","./all_0.1_sim_train_NN/batch5_sample1_data.npz"]
# files = ["./all_T60_0.1train_0inference/batch1_sample13_data.npz","./all_T60_0.1train_0inference/batch2_sample2_data.npz"]



for file_path in files:
    data = np.load(file_path)

    print(f"\nVisualizing: {file_path}")

    np.set_printoptions(threshold=np.inf, linewidth=200)
    # print(data['mean_out2']) #MC Dropout Mean
    print(data['std_out2']) # uncertainty std
    # visualize_single_cloud(data['src_pcd'], "Source Partial Input", color=[1, 0, 0])
    # visualize_single_cloud(data['src_pcd_inter'], "Intermediate Output", color=[0, 0, 1])
    visualize_single_cloud(data['xyz'], "Final Output Prediction", color=[0, 1, 0])
    # visualize_single_cloud(data['model_pcd_transformed'], "Ground Truth", color=[0, 0, 0])
    visualize_single_cloud(data['mean_out2'], "MC Dropout Mean", color=[1, 0.5, 0])



# rank std of different pointclouds

In [ ]:
import os
import numpy as np
import glob
import open3d as o3d
import matplotlib.pyplot as plt



def visualize_prediction_with_gt_and_input(
    pred_xyz_mc=None,
    pred_xyz_single=None,
    gt_xyz=None,
    partial_xyz=None,
    title="",
    show_mc=True,
    show_single=True,
    show_gt=True,
    show_input=True
):
    geoms = []

    if show_mc and pred_xyz_mc is not None:
        pcd_mc = o3d.geometry.PointCloud()
        pcd_mc.points = o3d.utility.Vector3dVector(pred_xyz_mc)
        pcd_mc.paint_uniform_color([1, 0, 0])  # Red
        geoms.append(pcd_mc)

    if show_single and pred_xyz_single is not None:
        pcd_single = o3d.geometry.PointCloud()
        pcd_single.points = o3d.utility.Vector3dVector(pred_xyz_single)
        pcd_single.paint_uniform_color([1, 0.5, 0])  # Orange
        geoms.append(pcd_single)

    if show_gt and gt_xyz is not None:
        pcd_gt = o3d.geometry.PointCloud()
        pcd_gt.points = o3d.utility.Vector3dVector(gt_xyz)
        pcd_gt.paint_uniform_color([0, 1, 0])  # Green
        geoms.append(pcd_gt)

    if show_input and partial_xyz is not None:
        pcd_input = o3d.geometry.PointCloud()
        pcd_input.points = o3d.utility.Vector3dVector(partial_xyz)
        pcd_input.paint_uniform_color([0, 0.5, 1])  # Blue
        geoms.append(pcd_input)

    if len(geoms) == 0:
        print("⚠️ Nothing selected for visualization.")
        return

    o3d.visualization.draw_geometries(geoms, window_name=title)


def rank_strawberries(npz_folder):
    all_files = sorted(glob.glob(os.path.join(npz_folder, '*.npz')))
    rankings = []

    for f in all_files:
        data = np.load(f)

        std_out2 = data['std_out2']  # (N, 3)
        if std_out2.ndim == 2:
            per_point_std = np.linalg.norm(std_out2, axis=1)  # Shape (N,)
        else:
            per_point_std = std_out2.squeeze()

        overall_std = per_point_std.mean()
        rankings.append((f, overall_std))

    rankings.sort(key=lambda x: x[1])  # Ascending = Most certain first
    return rankings


folder = 'all_T60_0.1' #'all_0.1_sim_train_NN'  
rankings = rank_strawberries(folder)

print("\n🍓 Ranked Predictions by Certainty (lower = more confident):")
for i, (f, score) in enumerate(rankings):
    print(f"{i+1:2d}. {os.path.basename(f)} - Mean STD: {score:.6f}")

# Visualize most and least certain
most_certain_file = rankings[0][0]
least_certain_file = rankings[-1][0]

# most_certain_file = "./all_T60_0.1train_0inference/batch1_sample13_data.npz"
# least_certain_file = "./all_T60_0.1train_0inference/batch2_sample2_data.npz"


data = np.load(most_certain_file)
visualize_prediction_with_gt_and_input(
    pred_xyz_mc=data['mean_out2'],
    pred_xyz_single=data['xyz'],
    gt_xyz=data['model_pcd_transformed'],
    partial_xyz=data['src_pcd'],
    title="Most Certain Prediction",
    show_mc=True,
    show_single=True,
    show_gt=True,
    show_input=True
)


data = np.load(least_certain_file)
visualize_prediction_with_gt_and_input(
    pred_xyz_mc=data['mean_out2'],
    pred_xyz_single=data['xyz'],
    gt_xyz=data['model_pcd_transformed'],
    partial_xyz=data['src_pcd'],
    title="Least Certain Prediction",
    show_mc=True,
    show_single=True,
    show_gt=True,
    show_input=True
)



